In [4]:
import os
import cv2
import torch
from ultralytics import YOLO
import logging
from pathlib import Path
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class FieldExtractor:
    def __init__(self, model_path, output_dir=None):
        # Class names matching your training
        self.classes = [
            'AWB-No', 'FOB-Value-in-INR', 'Goods-Description', 
            'Non-GST-Invoice-No', 'Qty', 'Total-Value', 'Unit-Value'
        ]
        
        # Store paths
        self.model_path = model_path
        # Set output directory to local 'inference' folder if not specified
        self.output_base_dir = output_dir if output_dir else os.path.join(os.getcwd(), 'inference')
        
        # Create output directory if it doesn't exist
        os.makedirs(self.output_base_dir, exist_ok=True)
        
        # Load the model
        self.load_model()
        
    def load_model(self):
        """Load the YOLOv8 model"""
        try:
            self.model = YOLO(self.model_path)
            # Move to GPU if available
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            self.model.to(self.device)
            logger.info(f"Model loaded successfully on {self.device}")
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise
            
    def process_image(self, image_path, conf_threshold=0.25):
        """
        Process a single image and extract fields
        
        Args:
            image_path (str): Path to the input image
            conf_threshold (float): Confidence threshold for detections
            
        Returns:
            str: Path to the output directory containing cropped fields
        """
        try:
        
            output_dir = os.path.join(
                self.output_base_dir,
                f"{Path(image_path).stem}_jpg"
            )
            os.makedirs(output_dir, exist_ok=True)
            
            # Read image
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Could not read image: {image_path}")
            
            # Get predictions
            results = self.model(image)[0]
            
            # Process each detection
            detections_count = 0
            for det in results.boxes.data:
                x1, y1, x2, y2, conf, cls = det
                
                # Skip low confidence detections
                if conf < conf_threshold:
                    continue
                    
                # Convert coordinates to integers
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                class_name = self.classes[int(cls)]
                
                # Crop the field
                field_crop = image[y1:y2, x1:x2]
                
                # Save cropped image
                output_path = os.path.join(output_dir, f"{class_name}.jpg")
                cv2.imwrite(output_path, field_crop)
                
                detections_count += 1
                logger.info(f"Saved {class_name} (conf: {conf:.2f}) to {output_path}")
            
            logger.info(f"Processed {image_path}: Found {detections_count} fields")
            return output_dir
            
        except Exception as e:
            logger.error(f"Error processing image {image_path}: {str(e)}")
            raise

def main():
    # Define paths (modify these according to your setup)
    model_path = "best.pt"  # Path to your trained model
    image_path = "1.jpg"  # Path to your test image
    
    # Initialize the extractor (without specifying output_dir to use local inference folder)
    try:
        extractor = FieldExtractor(model_path)
        logger.info("Model initialized successfully")
    except Exception as e:
        logger.error(f"Failed to initialize model: {str(e)}")
        return
    
    # Process the image
    try:
        output_dir = extractor.process_image(image_path)
        print(f"\nProcessing completed successfully!")
        print(f"Cropped fields saved to: {output_dir}")
    except Exception as e:
        print(f"\nError during processing: {str(e)}")

if __name__ == "__main__":
    main()

2025-01-19 12:01:38,155 - INFO - Model loaded successfully on cpu
2025-01-19 12:01:38,156 - INFO - Model initialized successfully



0: 480x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 48.0ms
Speed: 3.0ms preprocess, 48.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


2025-01-19 12:01:38,282 - INFO - Saved Total-Value (conf: 0.96) to d:\Github_Projects\VTM_Final_Src_Code_Git_Connected\inference\1_jpg\Total-Value.jpg
2025-01-19 12:01:38,284 - INFO - Saved Qty (conf: 0.91) to d:\Github_Projects\VTM_Final_Src_Code_Git_Connected\inference\1_jpg\Qty.jpg
2025-01-19 12:01:38,286 - INFO - Saved Non-GST-Invoice-No (conf: 0.91) to d:\Github_Projects\VTM_Final_Src_Code_Git_Connected\inference\1_jpg\Non-GST-Invoice-No.jpg
2025-01-19 12:01:38,288 - INFO - Saved FOB-Value-in-INR (conf: 0.90) to d:\Github_Projects\VTM_Final_Src_Code_Git_Connected\inference\1_jpg\FOB-Value-in-INR.jpg
2025-01-19 12:01:38,290 - INFO - Saved Unit-Value (conf: 0.90) to d:\Github_Projects\VTM_Final_Src_Code_Git_Connected\inference\1_jpg\Unit-Value.jpg
2025-01-19 12:01:38,294 - INFO - Saved Goods-Description (conf: 0.82) to d:\Github_Projects\VTM_Final_Src_Code_Git_Connected\inference\1_jpg\Goods-Description.jpg
2025-01-19 12:01:38,297 - INFO - Saved AWB-No (conf: 0.64) to d:\Github_Proj


Processing completed successfully!
Cropped fields saved to: d:\Github_Projects\VTM_Final_Src_Code_Git_Connected\inference\1_jpg
